In [1]:
import warnings; warnings.simplefilter('ignore')
# Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pandas_profiling as prof
import pivottablejs as pvt
import datetime as dt
import os
%matplotlib inline

> first forecast for total_receipt quantities, and then forecast for the total issue quantities (Demand) 
> Get the over stock, under stock from them get stock coefficient
> now send the Stock coeff to Custom loss function and if there was an under stock increase the stock coeff(this should be in positive), where there was an over stock decrease the stock coeff (this should be in negative)
> once we add both, the Demand will get adjusted as per the weights now check the Loss function value
> it should not be high and should not be low (High value leads to Over stock again, Low value leads to repeat of same Demand so it should be moderate)

In [2]:
df=pd.read_excel(r'C:\Users\ashok.swarna\OneDrive - Accenture\BOSCH\random_inventory_data.xlsx')

In [3]:
df.head()

,Batch,Material,To Date,agg_opening_stock,total_receipt_quantities,agg_total_issue_quantities,agg_closing_stock,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,101,M1,2018-01-01,409,288,-667,30,0,30,0
1,101,M1,2018-01-02,30,220,-161,89,0,89,0
2,101,M1,2018-01-03,89,397,-250,236,0,236,0
3,101,M1,2018-01-04,236,198,-361,73,0,73,0
4,101,M1,2018-01-05,73,534,-419,188,0,188,0


In [4]:
df.Material.value_counts()

M1    365
Name: Material, dtype: int64

In [5]:
df_1=df[['Material','To Date','agg_opening_stock','total_receipt_quantities','agg_closing_stock','agg_total_issue_quantities']]

In [6]:
df_1.head()

,Material,To Date,agg_opening_stock,total_receipt_quantities,agg_closing_stock,agg_total_issue_quantities
0,M1,2018-01-01,409,288,30,-667
1,M1,2018-01-02,30,220,89,-161
2,M1,2018-01-03,89,397,236,-250
3,M1,2018-01-04,236,198,73,-361
4,M1,2018-01-05,73,534,188,-419


In [7]:
#df_1=df[df['Material']=='M303.160.117']

In [8]:
df_1['agg_total_issue_quantities']=abs(df_1.agg_total_issue_quantities)

In [9]:
df_1.head()

,Material,To Date,agg_opening_stock,total_receipt_quantities,agg_closing_stock,agg_total_issue_quantities
0,M1,2018-01-01,409,288,30,667
1,M1,2018-01-02,30,220,89,161
2,M1,2018-01-03,89,397,236,250
3,M1,2018-01-04,236,198,73,361
4,M1,2018-01-05,73,534,188,419


In [10]:
df_1.drop(['To Date','Material'], axis=1, inplace=True)

In [11]:
df_1.head()

,agg_opening_stock,total_receipt_quantities,agg_closing_stock,agg_total_issue_quantities
0,409,288,30,667
1,30,220,89,161
2,89,397,236,250
3,236,198,73,361
4,73,534,188,419


In [12]:
df_1.agg_closing_stock.value_counts()

0       24
32       8
189      7
470      6
370      5
188      4
24       4
500      4
53       4
209      3
180      3
300      3
306      3
58       3
270      3
159      3
135      3
103      3
224      3
100      3
88       3
235      3
590      3
60       3
89       3
23       3
2        3
18       3
46       3
33       3
        ..
1148     1
1386     1
244      1
106      1
108      1
1136     1
113      1
1138     1
700      1
117      1
630      1
119      1
120      1
121      1
350      1
318      1
777      1
129      1
339      1
335      1
800      1
650      1
139      1
331      1
148      1
328      1
153      1
157      1
670      1
580      1
Name: agg_closing_stock, Length: 208, dtype: int64

In [13]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [14]:
df_1_np=series_to_supervised(df_1[['agg_total_issue_quantities']].values,3)

In [15]:
df_1_np.head()

,var1(t-3),var1(t-2),var1(t-1),var1(t)
3,667.0,161.0,250.0,361
4,161.0,250.0,361.0,419
5,250.0,361.0,419.0,631
6,361.0,419.0,631.0,292
7,419.0,631.0,292.0,280


In [16]:
df_1_np.describe()

,var1(t-3),var1(t-2),var1(t-1),var1(t)
count,362.000000,362.000000,362.000000,362.000000
mean,294.071823,292.314917,291.944751,291.372928
std,299.805958,299.477838,299.723311,300.000889
min,0.000000,0.000000,0.000000,0.000000
25%,100.000000,100.000000,100.000000,100.000000
50%,234.000000,232.500000,232.500000,231.500000
75%,400.000000,400.000000,400.000000,400.000000
max,3027.000000,3027.000000,3027.000000,3027.000000


In [17]:
X_df = df_1_np.iloc[:, :-1]

In [18]:
X_df.head()

,var1(t-3),var1(t-2),var1(t-1)
3,667.0,161.0,250.0
4,161.0,250.0,361.0
5,250.0,361.0,419.0
6,361.0,419.0,631.0
7,419.0,631.0,292.0


In [19]:
y_df = df_1_np.iloc[:, 3]

In [20]:
y_df.head()

3    361
4    419
5    631
6    292
7    280
Name: var1(t), dtype: int64

In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import keras.losses
import tensorflow as tf
import keras.backend as K

Using TensorFlow backend.


In [22]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [23]:
classifier = Sequential()
classifier.add(Dense(output_dim = 3, init='uniform', activation = 'relu', input_dim = 3))
classifier.add(Dense(output_dim = 3, init='uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 1, init='uniform', activation = 'linear'))
classifier.compile(optimizer = 'adam', loss ='mean_squared_error')

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  after removing the cwd from sys.path.


In [24]:
classifier.fit(X_df, y_df, batch_size=32, nb_epoch = 1000)

Instructions for updating:
Use tf.cast instead.


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/1000
362/362 [==============================] - 1s 2ms/step - loss: 174638.0079
Epoch 2/1000
362/362 [==============================] - 0s 63us/step - loss: 174603.0186
Epoch 3/1000
362/362 [==============================] - 0s 63us/step - loss: 174526.5509
Epoch 4/1000
362/362 [==============================] - 0s 55us/step - loss: 174370.0565
Epoch 5/1000
362/362 [==============================] - 0s 55us/step - loss: 174062.6591
Epoch 6/1000
362/362 [==============================] - 0s 52us/step - loss: 173538.5085
Epoch 7/1000
362/362 [==============================] - 0s 52us/step - loss: 172738.5465
Epoch 8/1000
362/362 [==============================] - 0s 55us/step - loss: 171542.6212
Epoch 9/1000
362/362 [==============================] - 0s 52us/step - loss: 169773.5918
Epoch 10/1000
362/362 [==============================] - 0s 52us/step - loss: 167532.6094
Epoch 11/1000
362/362 [==============================] - 0s 52us/step - loss: 164603.9451
Epoch 12/1000
362/36

362/362 [==============================] - 0s 44us/step - loss: 80795.8441
Epoch 184/1000
362/362 [==============================] - 0s 50us/step - loss: 80766.4430
Epoch 185/1000
362/362 [==============================] - 0s 47us/step - loss: 80778.4739
Epoch 186/1000
362/362 [==============================] - 0s 50us/step - loss: 80804.4862
Epoch 187/1000
362/362 [==============================] - 0s 55us/step - loss: 80801.9013
Epoch 188/1000
362/362 [==============================] - 0s 44us/step - loss: 80730.4364
Epoch 189/1000
362/362 [==============================] - 0s 52us/step - loss: 80723.2396
Epoch 190/1000
362/362 [==============================] - 0s 50us/step - loss: 80749.2958
Epoch 191/1000
362/362 [==============================] - 0s 47us/step - loss: 80730.3380
Epoch 192/1000
362/362 [==============================] - 0s 47us/step - loss: 80721.0095
Epoch 193/1000
362/362 [==============================] - 0s 52us/step - loss: 80751.5857
Epoch 194/1000
362/362 [=

362/362 [==============================] - 0s 50us/step - loss: 80408.7589
Epoch 364/1000
362/362 [==============================] - 0s 47us/step - loss: 80398.7228
Epoch 365/1000
362/362 [==============================] - 0s 39us/step - loss: 80395.5043
Epoch 366/1000
362/362 [==============================] - 0s 52us/step - loss: 80376.9625
Epoch 367/1000
362/362 [==============================] - 0s 41us/step - loss: 80401.4549
Epoch 368/1000
362/362 [==============================] - 0s 47us/step - loss: 80393.2941
Epoch 369/1000
362/362 [==============================] - 0s 47us/step - loss: 80400.8998
Epoch 370/1000
362/362 [==============================] - 0s 52us/step - loss: 80384.9136
Epoch 371/1000
362/362 [==============================] - 0s 44us/step - loss: 80441.8786
Epoch 372/1000
362/362 [==============================] - 0s 47us/step - loss: 80429.2190
Epoch 373/1000
362/362 [==============================] - 0s 50us/step - loss: 80419.2794
Epoch 374/1000
362/362 [=

362/362 [==============================] - 0s 58us/step - loss: 80121.3628
Epoch 546/1000
362/362 [==============================] - 0s 66us/step - loss: 80119.9638
Epoch 547/1000
362/362 [==============================] - 0s 63us/step - loss: 80093.1208
Epoch 548/1000
362/362 [==============================] - 0s 69us/step - loss: 80210.3542
Epoch 549/1000
362/362 [==============================] - 0s 72us/step - loss: 80096.1952
Epoch 550/1000
362/362 [==============================] - 0s 72us/step - loss: 80120.3847
Epoch 551/1000
362/362 [==============================] - 0s 80us/step - loss: 80101.5245
Epoch 552/1000
362/362 [==============================] - 0s 74us/step - loss: 80097.3939
Epoch 553/1000
362/362 [==============================] - 0s 66us/step - loss: 80087.3962
Epoch 554/1000
362/362 [==============================] - 0s 61us/step - loss: 80085.2589
Epoch 555/1000
362/362 [==============================] - 0s 58us/step - loss: 80147.2399
Epoch 556/1000
362/362 [=

362/362 [==============================] - 0s 50us/step - loss: 79848.2988
Epoch 727/1000
362/362 [==============================] - 0s 44us/step - loss: 79800.8148
Epoch 728/1000
362/362 [==============================] - 0s 41us/step - loss: 79804.6256
Epoch 729/1000
362/362 [==============================] - 0s 61us/step - loss: 79813.5456
Epoch 730/1000
362/362 [==============================] - 0s 63us/step - loss: 79795.0249
Epoch 731/1000
362/362 [==============================] - 0s 47us/step - loss: 79799.3604
Epoch 732/1000
362/362 [==============================] - 0s 39us/step - loss: 79820.3383
Epoch 733/1000
362/362 [==============================] - 0s 47us/step - loss: 79780.7907
Epoch 734/1000
362/362 [==============================] - 0s 47us/step - loss: 79823.4273
Epoch 735/1000
362/362 [==============================] - 0s 50us/step - loss: 79887.7899
Epoch 736/1000
362/362 [==============================] - 0s 47us/step - loss: 79939.0514
Epoch 737/1000
362/362 [=

362/362 [==============================] - 0s 47us/step - loss: 79512.0417
Epoch 908/1000
362/362 [==============================] - 0s 47us/step - loss: 79522.5338
Epoch 909/1000
362/362 [==============================] - 0s 41us/step - loss: 79511.1728
Epoch 910/1000
362/362 [==============================] - 0s 61us/step - loss: 79537.1405
Epoch 911/1000
362/362 [==============================] - 0s 47us/step - loss: 79509.6324
Epoch 912/1000
362/362 [==============================] - 0s 47us/step - loss: 79533.5973
Epoch 913/1000
362/362 [==============================] - 0s 47us/step - loss: 79491.0278
Epoch 914/1000
362/362 [==============================] - 0s 47us/step - loss: 79557.5878
Epoch 915/1000
362/362 [==============================] - 0s 44us/step - loss: 79510.3187
Epoch 916/1000
362/362 [==============================] - 0s 58us/step - loss: 79528.9102
Epoch 917/1000
362/362 [==============================] - 0s 61us/step - loss: 79504.5159
Epoch 918/1000
362/362 [=

In [25]:
y_prediction=classifier.predict(X_df.values)

In [26]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

In [27]:
rms = sqrt(mean_squared_error(y_df, y_prediction))
rms

281.6997693522274

In [28]:
score = r2_score(y_df, y_prediction)  # First R-Squared
score

0.115843250160925

In [29]:
df_1_y=df_1.iloc[3:,:].copy()
df_1_y['Forecast']=y_prediction.astype(int)

In [30]:
df_1_y['over_stock']=np.where(df_1_y.agg_closing_stock> 0, df_1_y.agg_closing_stock, 0)

In [31]:
df_1_y['under_stock']=np.where(df_1_y.agg_closing_stock == 0, -(df_1_y.Forecast-df_1_y.agg_total_issue_quantities), 0)
df_1_y['under_stock']=np.where(df_1_y.under_stock >0 , 0, df_1_y.under_stock)

In [32]:
df_1_y

,agg_opening_stock,total_receipt_quantities,agg_closing_stock,agg_total_issue_quantities,Forecast,over_stock,under_stock
3,236,198,73,361,229,73,0
4,73,534,188,419,235,188,0
5,188,546,103,631,303,103,0
6,103,413,224,292,401,224,0
7,224,335,279,280,379,279,0
8,279,371,590,60,280,590,0
9,590,286,402,474,168,402,0
10,402,349,459,292,216,459,0
11,459,497,507,449,281,507,0
12,507,346,398,455,314,398,0


In [33]:
df_1_y['stock_coef']=np.where(df_1_y.over_stock == 0 , df_1_y.under_stock, df_1_y.over_stock )
df_1_y

,agg_opening_stock,total_receipt_quantities,agg_closing_stock,agg_total_issue_quantities,Forecast,over_stock,under_stock,stock_coef
3,236,198,73,361,229,73,0,73
4,73,534,188,419,235,188,0,188
5,188,546,103,631,303,103,0,103
6,103,413,224,292,401,224,0,224
7,224,335,279,280,379,279,0,279
8,279,371,590,60,280,590,0,590
9,590,286,402,474,168,402,0,402
10,402,349,459,292,216,459,0,459
11,459,497,507,449,281,507,0,507
12,507,346,398,455,314,398,0,398


In [34]:
df_1_y.sum()

agg_opening_stock              95143
total_receipt_quantities      105241
agg_closing_stock              94907
agg_total_issue_quantities    105477
Forecast                       88467
over_stock                     94907
under_stock                     -696
stock_coef                     94211
dtype: int64

In [35]:
#df_for_loss = df_1[['agg_closing_stock','agg_total_issue_quantities']]
df_for_loss = df_1_y[['stock_coef']].copy()
df_for_loss = df_for_loss.reset_index(drop=True)

In [36]:
df_for_loss.head()

,stock_coef
0,73
1,188
2,103
3,224
4,279


In [37]:
df_for_loss['os_flag']=np.where(df_for_loss.stock_coef > 0, 1, 0)

In [38]:
df_for_loss.head()

,stock_coef,os_flag
0,73,1
1,188,1
2,103,1
3,224,1
4,279,1


In [39]:
#normalizing between 1 to 2
df_for_loss.stock_coef=((df_for_loss.stock_coef-df_for_loss.stock_coef.min())/(df_for_loss.stock_coef.max()-df_for_loss.stock_coef.min()))*(2-(1))+(1) 

In [40]:
df_for_loss.head()

,stock_coef,os_flag
0,1.121212,1
1,1.176527,1
2,1.135642,1
3,1.193843,1
4,1.220298,1


In [41]:
#adding sign based on over stock and under stock
df_for_loss['stock_coef'] = np.where(df_for_loss.os_flag == 1, df_for_loss.stock_coef, (-1)*df_for_loss.stock_coef)

In [42]:
df_for_loss.head()

,stock_coef,os_flag
0,1.121212,1
1,1.176527,1
2,1.135642,1
3,1.193843,1
4,1.220298,1


In [43]:
def Custom_loss2(y_true, y_pred, df_for_loss, ch, cs, sample_weight=None, multioutput='uniform_average'):
    df=df_for_loss.copy()
    #whenever there is over stock, we are multiplying the coef with ch
    df['stock_coef'] = np.where(df.stock_coef > 0, ch*df.stock_coef, df.stock_coef)
    #whenever there is under stock, we are multiplying it with cs
    df['stock_coef'] = np.where(df.stock_coef < 0, cs*df.stock_coef, df.stock_coef)
    df.stock_coef= df.stock_coef.astype('float32', raise_on_error = False)
    #normalization
    #df.stock_coef=((df.stock_coef-df.stock_coef.min())/(df.stock_coef.max()-df.stock_coef.min()))*(1-(-1))+(-1) 
    #df.stock_coef=((df.stock_coef-df.stock_coef.min())/(df.stock_coef.max()-df.stock_coef.min()))
    data=tf.convert_to_tensor(df.stock_coef)
    #y_pred=K.tf.multiply(y_pred,data)
    y_pred=K.tf.add(y_pred,data)
    return K.mean(K.square(y_pred - y_true), axis=-1)

In [44]:
def custom_loss(df_for_loss, ch, cs):
  def loss_temp(y_true, y_pred):
    return Custom_loss2(y_true,y_pred, df_for_loss, ch, cs)
  return loss_temp

In [45]:
keras.losses.custom_loss = custom_loss

In [46]:
def model_compile(df_for_loss,ch,cs,X_df, y_df):
    classifier = Sequential()
    classifier.add(Dense(output_dim = 3, init='uniform', activation = 'relu', input_dim = 3))
    classifier.add(Dense(output_dim = 3, init='uniform', activation = 'relu'))
    classifier.add(Dense(output_dim = 1, init='uniform', activation = 'linear'))
    classifier.compile(optimizer = 'adam', loss = custom_loss(df_for_loss,ch,cs))
    classifier.fit(X_df, y_df, batch_size = 362, nb_epoch = 1000, verbose=0)
    return classifier

In [47]:
def df_generator(model,df_1_y):
    df_1_y_final=df_1_y.copy()
    y_prediction=model.predict(X_df)
    df_1_y_final['Opt_Inventory']=y_prediction.astype(int)
    df_1_y_final['pred_stock_coeff']=df_1_y_final['agg_opening_stock']+df_1_y_final['total_receipt_quantities']-df_1_y_final['Opt_Inventory']
    df_1_y_final['pred_over_stock'] = np.where(df_1_y_final['pred_stock_coeff'] > 0,df_1_y_final['pred_stock_coeff'], 0)
    df_1_y_final['pred_under_stock'] = np.where(df_1_y_final['pred_stock_coeff'] < 0,df_1_y_final['pred_stock_coeff'], 0)
    true_os=df_1_y_final['over_stock'].sum()
    true_us=df_1_y_final['under_stock'].sum()
    pred_os=df_1_y_final['pred_over_stock'].sum()
    pred_us=df_1_y_final['pred_under_stock'].sum()
    print('true over stock: %d; predicted over stock : %d; forecasted under stock: %d; predicted under stock: %d' % (true_os, pred_os, true_us, pred_us))
    #print(df_1_y_final[['over_stock','under_stock','pred_over_stock','pred_under_stock']])
    
    #return df_1_y_final
    df_1_y_final.to_csv('Bosch_Predicted_values.Csv', index=False)
    #save to excel####

In [48]:
#without normalized df_for_loss

In [49]:
model=model_compile(df_for_loss,1,1,X_df, y_df)
df_generator(model,df_1_y)

C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 200022; forecasted under stock: -696; predicted under stock: 0


In [50]:
Ch=[1.25,1.5,1.75,2]
Cs=[1]
for i in Ch:
    for j in Cs:
        print(" ")
        print('Ch %f; Cs %f'%(i,j))
        model=model_compile(df_for_loss,i,j,X_df, y_df)
        df_generator(model,df_1_y)

 
Ch 1.250000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116329; forecasted under stock: -696; predicted under stock: -3080
 
Ch 1.500000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116516; forecasted under stock: -696; predicted under stock: -3185
 
Ch 1.750000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116456; forecasted under stock: -696; predicted under stock: -3064
 
Ch 2.000000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116470; forecasted under stock: -696; predicted under stock: -3069


In [51]:
Ch=[1.05,1.10,1.15,1.2]
Cs=[1]
for i in Ch:
    for j in Cs:
        print(" ")
        print('Ch %f; Cs %f'%(i,j))
        model=model_compile(df_for_loss,i,j,X_df, y_df)
        df_generator(model,df_1_y)

 
Ch 1.050000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 200022; forecasted under stock: -696; predicted under stock: 0
 
Ch 1.100000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116294; forecasted under stock: -696; predicted under stock: -3088
 
Ch 1.150000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116333; forecasted under stock: -696; predicted under stock: -3089
 
Ch 1.200000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116311; forecasted under stock: -696; predicted under stock: -3089


In [52]:
#with normalized df_for_loss

In [53]:
model=model_compile(df_for_loss,1,1,X_df, y_df)
df_generator(model,df_1_y)

C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116273; forecasted under stock: -696; predicted under stock: -3091


In [54]:
Ch=[1.25,1.5,1.75,2]
Cs=[1]
for i in Ch:
    for j in Cs:
        print(" ")
        print('Ch %f; Cs %f'%(i,j))
        model=model_compile(df_for_loss,i,j,X_df, y_df)
        df_generator(model,df_1_y)

 
Ch 1.250000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116305; forecasted under stock: -696; predicted under stock: -3088
 
Ch 1.500000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116352; forecasted under stock: -696; predicted under stock: -3091
 
Ch 1.750000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116430; forecasted under stock: -696; predicted under stock: -3065
 
Ch 2.000000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116502; forecasted under stock: -696; predicted under stock: -3048


In [55]:
Ch=[1.05,1.10,1.15,1.2]
Cs=[1]
for i in Ch:
    for j in Cs:
        print(" ")
        print('Ch %f; Cs %f'%(i,j))
        model=model_compile(df_for_loss,i,j,X_df, y_df)
        df_generator(model,df_1_y)

 
Ch 1.050000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116196; forecasted under stock: -696; predicted under stock: -3114
 
Ch 1.100000; Cs 1.000000


C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=3, units=3, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=3, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1, kernel_initializer="uniform")`
  """
C:\Users\ashok.swarna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


true over stock: 94907; predicted over stock : 116191; forecasted under stock: -696; predicted under stock: -3122


PermissionError: [Errno 13] Permission denied: 'Bosch_Predicted_values.Csv'